In [29]:
import requests,json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



# Functions for getting SensorTower data. 
# Store_Intelligence
# Rate Limit: 6 requests per second. 
# Requests should be made to the following base URL, followed by the specific path to the API you want to use.
# Latest Updated Date: 2021.03.02
# Author: Chen

# Compact Download / Revenue Estimates ???


# Download / Revenue Estimates for applications (If run error please try https://api.sensortower-china.com:443)

# Input: 
# os: Operating System (use "ios", "android",or "unified")
# country: Checking CountryCodes_CategoryIDs to select country (use"WW" for worldwide, "US"  for United States)  
# date_granularity: use "daily", "weekly","monthly", or"quarterly",defaults to "daily"
# start_date:(yyyy-mm-dd) 
# end_date: (yyyy-mm-dd)  
# app_ids: (defaults to None) 
# publisher_ids: (defaults to None) 
# Do not give both app_ids and publisher_ids at same time. 

# Output: A dataframe which containz: 
 
# {
#   "sales_report_estimates_key": {
#     "ios": {
#       "aid": "App ID",
#       "cc": "Country Code",
#       "d": "Date",
#       "iu": "iPhone Downloads",
#       "ir": "iPhone Revenue",
#       "au": "iPad Downloads",
#       "ar": "iPad Revenue"
#     },
#     "android": {
#       "aid": "App ID",
#       "c": "Country Code",
#       "d": "Date",
#       "u": "Android Downloads",
#       "r": "Android Revenue"
#     },
#     "unified": {
#       "app_id": "App ID",
#       "country": "Country Code",
#       "date": "Date",
#       "android_units": "Android Downloads",
#       "android_revenue": "Android Revenue",
#       "ipad_units": "iPad Downloads",
#       "ipad_revenue": "iPad Revenue",
#       "iphone_units": "iPhone Downloads",
#       "iphone_revenue": "iPhone Revenue"
#     }
#   }
# }
# If used publisher_ids, all applications made by this publisher will return. 

# Example: 
# get_data_apps = requests_Download_RevenueEstimates('ios', 'US', 'daily', '2020-11-01', '2020-11-04', '1531465966')
# get_data_publishers = requests_Download_RevenueEstimates('ios', 'US', 'daily', '2020-11-01', '2020-11-04', publisher_ids = '1191495496') 

def requests_Download_RevenueEstimates(os, country, date_granularity, start_date, end_date, app_ids = None, publisher_ids = None):
    if app_ids == None and publisher_ids == None:
        return(print('Error, at least one app ID, or one publisher ID is required.'))
        
    elif publisher_ids == None:
        aux_DRE = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/sales_report_estimates?app_ids='+app_ids+
                               '&countries='+country+
                               '&date_granularity='+date_granularity+
                               '&start_date='+
                               start_date+'&end_date='+
                               end_date+'&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    else:
        aux_DRE = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/sales_report_estimates?publisher_ids='+publisher_ids+
                               '&countries='+country+
                               '&date_granularity='+date_granularity+
                               '&start_date='+
                               start_date+'&end_date='+
                               end_date+'&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_DRE = pd.DataFrame(aux_DRE.json())
    print(aux_DRE.status_code)
    return(df_DRE)


# Top Apps(If run error please try https://api.sensortower-china.com:443) for one day, week, month, quarter. 给出指定日期后，会寻找该日期所在的日，周，月，季度，
# Input:
# os: Operating System (use "ios", "android",or "unified")
# comparision_attribute: use "absolute", "delta", or "transformed_delta"
# time_range: use "day","week", "month", or "quarter"
# measure: use "units" or "revenue"
# device_type: use "iphone", "ipad", or "total" for `ios`, leave blank for `android`, use "total" for `unified`
# category: Checking CountryCodes_CategoryIDs to select category.(use 6014 for game applications)
# date: Date.YYYY-MM-DD format.Auto-changes to the beginning of time_range.Ex: Mondays for weeks, 1st of the month, 1st day of the quarter, 1st day of the year.
# country: Checking CountryCodes_CategoryIDs to select country (use"WW" for worldwide, "US"  for United States)
# limit: Limit how many apps per call. (Default: 25, Max: 2000)

# Output: A dataframe which contains informations of top Apps. 
# The variable custom_tags contains informations about each application. 

# Example_ios_uni: 
# https://api.sensortower-china.com/v1/ios/sales_report_estimates_comparison_attributes?comparison_attribute=absolute&time_range=week&measure=units&device_type=total&category=6014&date=2021-02-08&country=US&limit=25&auth_token=r7cTTJq5nBSqxMU_WmJo
# get_week_AppRanking1 = requests_ios_uni_TopApps_periodic('ios', 'absolute', 'week','units', 'total', '6014', '2021-02-08','US','25') # 2021-02-08---2021-02-14
# get_week_AppRanking2 = requests_ios_uni_TopApps_periodic('ios', 'absolute', 'week','units', 'total', '6014', '2021-02-07','US','25') # 2021-02-01---2021-02-7
# get_week_AppRanking3 = requests_ios_uni_TopApps_periodic('ios', 'absolute', 'week','units', 'total', '6014', '2021-02-03','US','25') # 2021-02-01---2021-02-7


# Example_Android: 
# https://api.sensortower-china.com/v1/android/sales_report_estimates_comparison_attributes?comparison_attribute=absolute&time_range=week&measure=units&category=game&date=2021-02-08&country=US&limit=25&auth_token=r7cTTJq5nBSqxMU_WmJo
# get_week_ranking4 = requests_Android_TopApps_periodic('android', 'absolute', 'week','units', 'game', '2021-02-08','US','25')

def requests_ios_uni_TopApps_periodic(os,comparision_attribute, time_range, measure, device_type, category, date, country, limit):  
    aux_TopApps = requests.get('https://api.sensortower-china.com:443/v1/'
                               +os+'/sales_report_estimates_comparison_attributes?comparison_attribute='+comparision_attribute+
                               '&time_range='+time_range+
                               '&measure='+measure+
                               '&device_type='+device_type+
                               '&category='+category+
                               '&date='+date+
                               '&country='+country+
                               '&limit='+limit+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_TopApps = pd.DataFrame(aux_TopApps.json())
    print(aux_TopApps.status_code) # Get response messages
    return(df_TopApps)


# def requests_TopApps_customized(os,comparision_attribute, measure, device_type, category, date, end_date, country, limit):  
#     aux_TopApps = requests.get('https://api.sensortower-china.com:443/v1/'
#                                +os+'/sales_report_estimates_comparison_attributes?comparison_attribute='+comparision_attribute+
#                                '&measure='+measure+
#                                '&device_type='+device_type+
#                                '&category='+category+
#                                '&date='+date+
#                                '&end_date='+end_date+
#                                '&country='+country+
#                                '&limit='+limit+
#                                '&auth_token=r7cTTJq5nBSqxMU_WmJo'
#                                )
#     df_TopApps = pd.DataFrame(aux_TopApps.json())
#     print(aux_TopApps.status_code) # Get response messages
#     return(df_TopApps)    


def requests_Android_TopApps_periodic(os,comparision_attribute, time_range, measure, category, date, country, limit):  
    aux_TopApps = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/sales_report_estimates_comparison_attributes?comparison_attribute='+comparision_attribute+
                               '&time_range='+time_range+
                               '&measure='+measure+
                               '&category='+category+
                               '&date='+date+
                               '&country='+country+
                               '&limit='+limit+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_TopApps = pd.DataFrame(aux_TopApps.json())
    print(aux_TopApps.status_code) # Get response messages
    return(df_TopApps)


# Top Publishers(If run error please try https://api.sensortower-china.com:443) for one day, week, month, quarter. 给出指定日期后，会寻找该日期所在的日，周，月，季度，
# Input:
# os: Operating System (use "ios", "android",or "unified")
# comparision_attribute: use "absolute", "delta", or "transformed_delta"
# time_range: use "day","week", "month", or "quarter"
# measure: use "units" or "revenue"
# device_type: use "iphone", "ipad", or "total" for `ios`, leave blank for `android` and `unified`
# category: Checking CountryCodes_CategoryIDs to select category.(use 6014 for game applications)
# date: Date.YYYY-MM-DD format.Auto-changes to the beginning of time_range.Ex: Mondays for weeks, 1st of the month, 1st day of the quarter, 1st day of the year.
# country: Checking CountryCodes_CategoryIDs to select country (use"WW" for worldwide, "US"  for United States)
# limit: Limit how many apps per call. (Default: 25, Max: 2000)

# Output: A dataframe which contains informations of top Publishers. 
# The variable apps contains all informations of applications made by each publisher. 

# Example_ios: 
# https://api.sensortower-china.com/v1/ios/top_and_trending/publishers?comparison_attribute=absolute&time_range=week&measure=units&device_type=total&category=6014&date=2021-02-08&country=US&limit=25&auth_token=r7cTTJq5nBSqxMU_WmJo
# get_week_PubRanking1_ios = requests_ios_TopPublishers_periodic('ios', 'absolute', 'week','units', 'total', '6014', '2021-02-08','US','25')

# Example_Android_uni: 
# https://api.sensortower-china.com/v1/android/top_and_trending/publishers?comparison_attribute=absolute&time_range=week&measure=units&category=6014&date=2021-02-08&country=US&limit=25&auth_token=r7cTTJq5nBSqxMU_WmJo
# get_week_PubRanking1_android = requests_Android_uni_TopPublishers_periodic('android', 'absolute', 'week','units', 'game', '2021-02-08','US','25')

def requests_ios_TopPublishers_periodic(os,comparision_attribute, time_range, measure, device_type, category, date, country, limit):  
    aux_TopPublishers = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/top_and_trending/publishers?comparison_attribute='+comparision_attribute+
                               '&time_range='+time_range+
                               '&measure='+measure+
                               '&device_type='+device_type+
                               '&category='+category+
                               '&date='+date+
                               '&country='+country+
                               '&limit='+limit+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_TopPublishers = pd.DataFrame(aux_TopPublishers.json())
    print(aux_TopPublishers.status_code) # Get response messages
    return(df_TopPublishers)


def requests_Android_uni_TopPublishers_periodic(os,comparision_attribute, time_range, measure, category, date, country, limit):  
    aux_TopPublishers = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/top_and_trending/publishers?comparison_attribute='+comparision_attribute+
                               '&time_range='+time_range+
                               '&measure='+measure+
                               '&category='+category+
                               '&date='+date+
                               '&country='+country+
                               '&limit='+limit+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_TopPublishers = pd.DataFrame(aux_TopPublishers.json())
    print(aux_TopPublishers.status_code) # Get response messages
    return(df_TopPublishers)



# Store Summary: Information for all ios or android.
# Input:
# Output: 


def requests_Store_Summary(os, category, country, granularity, start_date, end_date):  
    aux_SS = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/store_summary?categories='+category+
                               '&countries='+country+
                               '&date_granularity='+granularity+
                               '&start_date='+start_date+
                               '&end_date='+end_date+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_SS = pd.DataFrame(aux_SS.json())
    print(aux_SS.status_code) # Get response messages
    return(df_SS)

# Game Breakdown: Information for all ios or android.(games specific)
# Input:
# Output: 

def requests_Games_Breakdown(os, category, country, granularity, start_date, end_date):  
    aux_GB = requests.get('https://api.sensortower-china.com/v1/'
                               +os+'/games_breakdown?categories='+category+
                               '&countries='+country+
                               '&date_granularity='+granularity+
                               '&start_date='+start_date+
                               '&end_date='+end_date+
                               '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                               )
    df_GB = pd.DataFrame(aux_GB.json())
    print(aux_GB.status_code) # Get response messages
    return(df_GB)

# Reviews 
# Input: 
# os: ios or android
# app_id
# start_date
# end_date
# country: US or leave blank

# Output: 
# average: Rating
# total: Total number of person who rates.

# Example: 
# https://api.sensortower-china.com/v1/ios/review/get_ratings?app_id=284882215&start_date=2021-02-08&end_date=2021-02-14&country=US&auth_token=r7cTTJq5nBSqxMU_WmJo
# test_rating1 = requests_Rating('ios', '1540979366', '2020-01-01', '2021-03-16')
# test_rating2 = requests_Rating('android', 'com.sushirolls.app', '2020-01-01', '2021-03-16')

def requests_ios_Rating(os, app_id, start_date, end_date): 
        aux_rating = requests.get('https://api.sensortower-china.com/v1/'
                                   +os+'/review/get_ratings?app_id='+app_id+
                                   '&start_date='+start_date+
                                   '&end_date='+end_date+
                                   '&country=US&auth_token=r7cTTJq5nBSqxMU_WmJo'
                                   )
        df_rating = pd.DataFrame(aux_rating.json())
        print(aux_rating.status_code) # Get response messages
        return(df_rating)
        
def requests_android_Rating(os, app_id, start_date, end_date): 
            aux_rating = requests.get('https://api.sensortower-china.com/v1/'
                                       +os+'/review/get_ratings?app_id='+app_id+
                                       '&start_date='+start_date+
                                       '&end_date='+end_date+
                                       '&auth_token=r7cTTJq5nBSqxMU_WmJo'
                                       )
            df_rating = pd.DataFrame(aux_rating.json())
            print(aux_rating.status_code) # Get response messages
            return(df_rating)        
        
def requests_Rating(os):
    if os == 'ios':
        def aux_func(app_id, start_date, end_date):
            return requests_ios_Rating(os, app_id, start_date, end_date)
      
    else:
        def aux_func(app_id, start_date, end_date): 
            return requests_android_Rating(os, app_id, start_date, end_date)
        
    return aux_func    




In [30]:
test_rating1 = requests_Rating('ios')('1540979366', '2020-01-01', '2021-03-16')
test_rating1

200


,app_id,average,breakdown,country,current_version_breakdown,date,total
0,1540979366,4.430,"[7508, 2974, 5272, 8621, 77511]",US,"[7508, 2974, 5272, 8621, 77511]",2021-03-15T00:00:00Z,101886
1,1540979366,4.430,"[7498, 2973, 5263, 8612, 77449]",US,"[7498, 2973, 5263, 8612, 77449]",2021-03-14T00:00:00Z,101795
2,1540979366,4.429,"[7467, 2958, 5240, 8559, 76962]",US,"[7467, 2958, 5240, 8559, 76962]",2021-03-13T00:00:00Z,101186
3,1540979366,4.428,"[7446, 2951, 5214, 8524, 76624]",US,"[7446, 2951, 5214, 8524, 76624]",2021-03-12T00:00:00Z,100759
4,1540979366,4.429,"[7428, 2942, 5191, 8500, 76458]",US,"[7428, 2942, 5191, 8500, 76458]",2021-03-11T00:00:00Z,100519
5,1540979366,4.429,"[7395, 2936, 5173, 8462, 76134]",US,"[7395, 2936, 5173, 8462, 76134]",2021-03-09T00:00:00Z,100100
6,1540979366,4.428,"[7316, 2902, 5112, 8355, 75173]",US,"[7316, 2902, 5112, 8355, 75173]",2021-03-08T00:00:00Z,98858
7,1540979366,4.428,"[7275, 2882, 5078, 8308, 74707]",US,"[7275, 2882, 5078, 8308, 74707]",2021-03-06T00:00:00Z,98250
8,1540979366,4.427,"[7236, 2859, 5048, 8240, 74153]",US,"[7236, 2859, 5048, 8240, 74153]",2021-03-05T00:00:00Z,97536
9,1540979366,4.428,"[7189, 2839, 5023, 8211, 73824]",US,"[7189, 2839, 5023, 8211, 73824]",2021-03-03T00:00:00Z,97086


In [98]:
test_ss = requests_Store_Summary('ios', '6014','US','daily','2021-02-8','2021-02-14')
test_ss

200


,ar,au,ca,cc,d,ir,iu
0,570180771,1381311,6014,US,2021-02-08T00:00:00Z,1623085966,5145925
1,586174127,1406171,6014,US,2021-02-09T00:00:00Z,1741953621,5012830
2,633132247,1442601,6014,US,2021-02-10T00:00:00Z,1884967647,5121105
3,605893354,1479991,6014,US,2021-02-11T00:00:00Z,1926428128,5317180
4,762626495,2077744,6014,US,2021-02-12T00:00:00Z,2355910754,5745652
5,717461364,1957440,6014,US,2021-02-13T00:00:00Z,2007642594,6388467
6,732453326,1887731,6014,US,2021-02-14T00:00:00Z,1948838541,6326956


In [100]:
test_GB = requests_Games_Breakdown('ios', '6014','US','daily','2021-02-8','2021-02-14')
test_GB

200


,ar,au,ca,cc,d,ir,iu
0,570180771,1381311,6014,US,2021-02-08T00:00:00Z,1623085966,5145925
1,586174127,1406171,6014,US,2021-02-09T00:00:00Z,1741953621,5012830
2,633132247,1442601,6014,US,2021-02-10T00:00:00Z,1884967647,5121105
3,605893354,1479991,6014,US,2021-02-11T00:00:00Z,1926428128,5317180
4,762626495,2077744,6014,US,2021-02-12T00:00:00Z,2355910754,5745652
5,717461364,1957440,6014,US,2021-02-13T00:00:00Z,2007642594,6388467
6,732453326,1887731,6014,US,2021-02-14T00:00:00Z,1948838541,6326956
